# Find Game IDs

The objective of this notebook is to take the df with all team stats for each game and append the unique "game_id" to each game using nba_api.

In [ ]:
import pandas as pd
from nba_api.stats.static import teams
from nba_api.stats.endpoints import teamgamelog
import datetime

In [ ]:
PATH_TO_TEAM_DATA = "../../data/raw/nba_games_runtime.csv"

In [ ]:
df = pd.read_csv(PATH_TO_TEAM_DATA, index_col=0)
df

In [ ]:
# Create a "game_id" column and move to the front
df.insert(0, 'game_id', None)
df

In [ ]:
# Create a python dict to store team ids
team_ids = dict()

In [ ]:
# TODO: instead of sending an api request per game (over 20,000 requests), just send 30 requests of all games from 

# for index, row in df.iterrows(): 
    team_abbrev = row["team"]
    
    if(team_abbrev == "BRK"):
        team_abbrev = "BKN"

    curr_team_id = None

    if(team_abbrev not in team_ids):
        print(team_abbrev)
        curr_team_id = teams.find_team_by_abbreviation(team_abbrev)['id']
        print(f"{team_abbrev}: {curr_team_id}")
        team_ids[team_abbrev] = curr_team_id
    else:
        curr_team_id = team_ids[team_abbrev]


    # Format the game date into MM/DD/YY
    game_date = row["date"]
    formatted_date = datetime.datetime.strptime(game_date, "%Y-%m-%d").strftime("%m/%d/%Y")

    # Now we can fetch the gamelog for that team for that specific date
    gamelog = teamgamelog.TeamGameLog(
        season = 2015,
        season_type_all_star = 'Regular Season',
        team_id = curr_team_id,
        date_from_nullable = formatted_date,
        date_to_nullable = formatted_date
    )

    gamelog_df = gamelog.team_game_log.get_data_frame()
    assert gamelog_df.shape[0] == 1

    game_id = gamelog_df.iloc[0]["Game_ID"]
    row["game_id"] = game_id

In [ ]:
df[df["team"] == "CHO"] # Charlotte Hornets? Some of the abbrev are wrong!